In [5]:
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit, KFold, GridSearchCV, train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import accuracy_score,mean_squared_error
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
import datetime as dt

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter

import pyproj
import random
import seaborn as sns

global time_tuple
global time_tuple_plt
global g
g = pyproj.Geod(ellps='WGS84')

import warnings
warnings.filterwarnings('ignore')

In [9]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [7]:
def calculate_distance(lat1, lon1, lat2, lon2):
    earth_radius = 6371*1000  # m
    dlat = np.radians(lat2-lat1)
    dlon = np.radians(lon2-lon1)
    a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon/2) * np.sin(dlon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = earth_radius * c
    return d

def distance(train_data2):
    distance =  train_data2.apply(lambda x: calculate_distance(x['start_lat'],x['start_lng'], x['end_lat'], x['end_lng']),axis = 1, result_type = 'expand')
    train_data2 = pd.concat([train_data2, distance],axis = 1)
    train_data2.rename(columns = {0:'distance'}, inplace = True)
    return train_data2

def direction(lat1, lon1, lat2, lon2):
    heading = 0
    azN,azS,distance = g.inv(lon1, lat1, lon2, lat2)
    if azN <45 and azN > -45:
        heading = 'N'
    elif azS <45 and azS > -45:
        heading = 'S'
    elif azN >= 45 and azN < 90:
        heading = 'NE'
    elif azN >= -90  and azN <-45:
        heading = 'NW'
    elif azS >= 45 and azS < 90:
        heading = 'SW'
    elif azS >= -90  and azS <-45:
        heading = 'SE'
    return heading

def headings(all_features):
    az =  all_features.apply(lambda x: direction(x['start_lat'],x['start_lng'], x['end_lat'], x['end_lng']),axis = 1, result_type = 'expand')
    az.rename(columns = {0:'heading'}, inplace = True)
    all_features = pd.concat([all_features, az],axis = 1)
    all_features.rename(columns = {0:'heading'}, inplace = True)
    return all_features

def time(all_features):
    all_features['datetime'] = pd.to_datetime(all_features['datetime'])
    all_features['hour'] = pd.DatetimeIndex(all_features['datetime']).hour
    all_features['month'] = pd.DatetimeIndex(all_features['datetime']).month
    all_features['year'] = pd.DatetimeIndex(all_features['datetime']).year
    all_features['weekday'] = all_features['datetime'].dt.dayofweek
    return 0

def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        j = np.array(indices[i: min(i + batch_size, num_examples)])
        if len(np.take(features,j, axis = 0)) >= batch_size:
            yield np.take(features,j, axis = 0), np.take(labels,j, axis = 0)
            
def drop(train_data2):
    #some invalid data drop out
    train_data2['speed'] = train_data2['distance']/train_data['duration']
    train_data2 = train_data2[train_data2['duration'] <= 15000]
    train_data2 = train_data2[train_data2['speed'] > 1]
    train_data2 = train_data2[train_data2['speed'] < 37]
    train_data2.reset_index(drop=True)
    train_data2 = train_data2.drop(columns = ['speed'])
    train_data2 = train_data2.dropna()
    return train_data2


#PLOTTING ===========================
def logDuration(trainNYC):
    trainNYC['log_duration'] = np.log(trainNYC['duration'].values + 1)
    return trainNYC
    
def plotDuration(trainNYC):
    plt.hist(trainNYC['duration'].values, bins=100)
    plt.xlabel('trip_duration')
    plt.ylabel('number of train records')
    plt.show()
    sns.distplot(trainNYC["log_duration"], bins =100)
    
    
#PCA and CLUSTER ==============================
def pcaTrans(train, test):
    coords = np.vstack((train[['start_lat', 'start_lng']].values,
                        train[['end_lat', 'end_lng']].values,
                        test[['start_lat', 'start_lng']].values,
                        test[['end_lat', 'end_lng']].values))

    pca = PCA().fit(coords)
    train['pickup_pca0'] = pca.transform(train[['start_lat', 'start_lng']])[:, 0]
    train['pickup_pca1'] = pca.transform(train[['start_lat', 'start_lng']])[:, 1]
    train['dropoff_pca0'] = pca.transform(train[['end_lat', 'end_lng']])[:, 0]
    train['dropoff_pca1'] = pca.transform(train[['end_lat', 'end_lng']])[:, 1]
    test['pickup_pca0'] = pca.transform(test[['start_lat', 'start_lng']])[:, 0]
    test['pickup_pca1'] = pca.transform(test[['start_lat', 'start_lng']])[:, 1]
    test['dropoff_pca0'] = pca.transform(test[['end_lat', 'end_lng']])[:, 0]
    test['dropoff_pca1'] = pca.transform(test[['end_lat', 'end_lng']])[:, 1]
    return train, test, coords


def findCluster(train, test, kmeans):
    num = Counter(kmeans.labels_)
    train.loc[:, 'pickup_cluster'] = kmeans.predict(train[['start_lat', 'start_lng']])
    train.loc[:, 'dropoff_cluster'] = kmeans.predict(train[['end_lat', 'end_lng']])
    test.loc[:, 'pickup_cluster'] = kmeans.predict(test[['start_lat', 'start_lng']])
    test.loc[:, 'dropoff_cluster'] = kmeans.predict(test[['end_lat', 'end_lng']])
   
    train_pickup =  train.apply(lambda x: num[x['pickup_cluster']],axis = 1, result_type = 'expand')
    train = pd.concat([train, train_pickup],axis = 1)
    train.rename(columns = {0:'pickup_neighbor'}, inplace = True)
    
    train_dropoff = train.apply(lambda x: num[x['dropoff_cluster']],axis = 1, result_type = 'expand')
    train = pd.concat([train, train_dropoff],axis = 1)
    train.rename(columns = {0:'dropoff_neighbor'}, inplace = True)
    
    test_pickup =  test.apply(lambda x: num[x['pickup_cluster']],axis = 1, result_type = 'expand')
    test = pd.concat([test, test_pickup],axis = 1)
    test.rename(columns = {0:'pickup_neighbor'}, inplace = True)
    
    test_dropoff = test.apply(lambda x: num[x['dropoff_cluster']],axis = 1, result_type = 'expand')
    test = pd.concat([test, test_dropoff],axis = 1)
    test.rename(columns = {0:'dropoff_neighbor'}, inplace = True)
    
    return train, test

In [10]:
#calculate distance
train_data = distance(train_data)
test_data = distance(test_data)

#calculate heading
train_data = headings(train_data)
test_data = headings(test_data)

#calculate time
time(train_data)
time(test_data)

#drop
train_data = drop(train_data)

#log
train= logDuration(train_data)
#test = logDuration(test_data)

#PCA coordinates
train, test, coords = pcaTrans(train_data, test_data)

#neighbor
kmeans = MiniBatchKMeans(n_clusters=200, batch_size=10000).fit(coords)

In [11]:
train, test = findCluster(train, test, kmeans)
train = train.drop(columns = ['datetime','year','start_lng','start_lat', 'end_lng', 'end_lat','pickup_cluster','dropoff_cluster'])
test = test.drop(columns = ['datetime','year','start_lng','start_lat', 'end_lng', 'end_lat','pickup_cluster','dropoff_cluster'])

In [12]:
train.head()

,row_id,duration,location,distance,heading,hour,month,weekday,log_duration,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1,pickup_neighbor,dropoff_neighbor
0,0,1815,2,15760.960867,NW,0,1,4,7.504392,-14.593484,-0.117743,-14.415553,-0.070122,2186,386
1,1,300,1,2535.404112,S,1,9,4,5.707110,34.129165,0.019897,34.128070,-0.002930,2798,3604
2,2,2620,2,9753.753494,SW,20,4,6,7.871311,-14.499547,0.016623,-14.384261,0.011995,269,2111
3,3,360,1,1168.773227,N,23,9,0,5.888878,34.108449,0.018188,34.117327,0.026221,2772,3440
4,4,582,2,1944.774374,N,12,1,2,6.368187,-14.372735,-0.034427,-14.376096,-0.017198,778,4458


In [13]:
test.head()

,row_id,location,distance,heading,hour,month,weekday,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1,pickup_neighbor,dropoff_neighbor
0,0,2,5401.617283,N,23,6,5,-14.380690,-0.023041,-14.422361,0.012797,4117,2849
1,1,1,2307.671358,NE,1,9,0,34.161733,0.006634,34.135600,0.007884,2053,1289
2,2,2,19580.248478,SE,11,9,5,-14.406939,0.000796,-14.586996,-0.115092,6327,1362
3,3,1,2970.366413,SW,5,9,0,34.113468,0.007667,34.138797,-0.009430,1247,1902
4,4,2,10089.006862,NE,9,2,3,-14.395431,0.000114,-14.512179,0.017080,3600,2261


In [14]:
heading = pd.get_dummies(train['heading'], prefix = 'h')
heading_test = pd.get_dummies(test['heading'], prefix = 'h')

In [15]:
train2 = pd.concat([train, heading], axis = 1)
test2 = pd.concat([test, heading_test], axis = 1)
train2.head()

,row_id,duration,location,distance,heading,hour,month,weekday,log_duration,pickup_pca0,...,dropoff_pca0,dropoff_pca1,pickup_neighbor,dropoff_neighbor,h_N,h_NE,h_NW,h_S,h_SE,h_SW
0,0,1815,2,15760.960867,NW,0,1,4,7.504392,-14.593484,...,-14.415553,-0.070122,2186,386,0,0,1,0,0,0
1,1,300,1,2535.404112,S,1,9,4,5.707110,34.129165,...,34.128070,-0.002930,2798,3604,0,0,0,1,0,0
2,2,2620,2,9753.753494,SW,20,4,6,7.871311,-14.499547,...,-14.384261,0.011995,269,2111,0,0,0,0,0,1
3,3,360,1,1168.773227,N,23,9,0,5.888878,34.108449,...,34.117327,0.026221,2772,3440,1,0,0,0,0,0
4,4,582,2,1944.774374,N,12,1,2,6.368187,-14.372735,...,-14.376096,-0.017198,778,4458,1,0,0,0,0,0


In [16]:
test2.head()

,row_id,location,distance,heading,hour,month,weekday,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1,pickup_neighbor,dropoff_neighbor,h_N,h_NE,h_NW,h_S,h_SE,h_SW
0,0,2,5401.617283,N,23,6,5,-14.380690,-0.023041,-14.422361,0.012797,4117,2849,1,0,0,0,0,0
1,1,1,2307.671358,NE,1,9,0,34.161733,0.006634,34.135600,0.007884,2053,1289,0,1,0,0,0,0
2,2,2,19580.248478,SE,11,9,5,-14.406939,0.000796,-14.586996,-0.115092,6327,1362,0,0,0,0,1,0
3,3,1,2970.366413,SW,5,9,0,34.113468,0.007667,34.138797,-0.009430,1247,1902,0,0,0,0,0,1
4,4,2,10089.006862,NE,9,2,3,-14.395431,0.000114,-14.512179,0.017080,3600,2261,0,1,0,0,0,0


In [17]:
train_features = train2.drop(columns =[ 'row_id','duration','heading','log_duration']).values
test_features = test2.drop(columns = ['row_id', 'heading']).values
train_labels = train2['log_duration'].values
train_labels = train_labels.reshape(-1,1)

In [18]:
print(train_features.shape)
print(test_features.shape)
print(train_labels.shape)

(137236, 17)
(30000, 17)
(137236, 1)


### try boosting

In [ ]:
for _ in range(5): # 10 passes through the data
    #clf = GradientBoostingRegressor(warm_start = True, max_depth = 8, n_estimators = 200, random_state = 23)
    clf = xgb.XGBRegressor(warm_start = True, max_depth = 15, n_estimators = 250, random_state = 23)
    train_iter = data_iter(90000, train_features, train_labels)
    train, valid = [], []

    print('==============epoch=================: ', _)
    
    
    for X, y in train_iter:
        #clf.fit(X,y)
        #print(X.shape)
        kf = KFold(n_splits=5, shuffle = True)

        for trn_idx, val_idx in kf.split(X):
        # split training data
            X_trn, X_tst, y_trn, y_tst = X[trn_idx], X[val_idx], y[trn_idx], y[val_idx] 

            clf.fit(X_trn, y_trn.ravel())
            trainpred = clf.predict(X_trn)
            #trainpred = np.exp(trainpred)-1
            
            train.append( mean_squared_error(y_trn, trainpred))

            pred = clf.predict(X_tst)
            #pred = np.exp(pred) - 1
            clf.n_estimators += 1 # increment by one so next  will add 1 tree
            valid.append(mean_squared_error(y_tst, pred))
    print('  Score train/valid: %.4f, %.4f' % (np.mean(np.sqrt(train)),np.mean(np.sqrt(valid))))

==============epoch=================:  0


In [20]:
predtest = clf.predict(test_features)
predtest = np.exp(predtest)-1
# reformat it for export to Kaggle
test['duration'] = pd.Series(predtest.reshape(1, -1)[0])
submission = pd.concat([test['row_id'], test['duration']], axis=1)
submission.to_csv('wait2.csv', index=False)

In [18]:
Train, Test = train_test_split(train2, test_size = 0.2, shuffle = True)
Train.shape, Test.shape

((111593, 20), (27899, 20))

In [19]:
X_train = Train.drop(columns =[ 'row_id','duration','heading'])
X_test = Test.drop(columns = ['row_id', 'heading','duration'])
Y_train = Train['duration']
Y_test = Test['duration']


dtrain = xgb.DMatrix(X_train, label=Y_train)
dvalid = xgb.DMatrix(X_test, label=Y_test)
dtest = xgb.DMatrix(test2.drop(columns =[ 'row_id','heading']))
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

In [20]:
xgb_pars = {'min_child_weight': 1, 'eta': 0.2, 'colsample_bytree': 0.8, 
            'max_depth': 10,
'subsample': 0.6, 'lambda': 3., 'nthread': -1, 'booster' : 'gbtree', 'silent': 1,
'eval_metric': 'rmse', 'objective': 'reg:linear'}
model = xgb.train(xgb_pars, dtrain, 50, watchlist, early_stopping_rounds=2,
      maximize=False, verbose_eval=1)
print('Modeling RMSE %.5f' % model.best_score)

[0]	train-rmse:850.26	valid-rmse:850.376
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 2 rounds.
[1]	train-rmse:727.182	valid-rmse:728.363
[2]	train-rmse:618.025	valid-rmse:620.192
[3]	train-rmse:535.414	valid-rmse:539.107
[4]	train-rmse:472.882	valid-rmse:478.24
[5]	train-rmse:427.461	valid-rmse:434.531
[6]	train-rmse:393.254	valid-rmse:402.984
[7]	train-rmse:367.656	valid-rmse:380.164
[8]	train-rmse:349.672	valid-rmse:364.223
[9]	train-rmse:335.791	valid-rmse:352.693
[10]	train-rmse:325.645	valid-rmse:344.854
[11]	train-rmse:317.896	valid-rmse:339.098
[12]	train-rmse:310.729	valid-rmse:334.515
[13]	train-rmse:304.987	valid-rmse:331.378
[14]	train-rmse:301.195	valid-rmse:329.074
[15]	train-rmse:297.739	valid-rmse:327.445
[16]	train-rmse:294.789	valid-rmse:325.903
[17]	train-rmse:292.344	valid-rmse:324.856
[18]	train-rmse:288.863	valid-rmse:323.685
[19]	train-rmse:286.483	valid-rmse:322.949
[20]	tra

In [21]:
pred = model.predict(dtest)
# reformat it for export to Kaggle
test2['duration'] = pd.Series(pred.reshape(1, -1)[0])
submission = pd.concat([test2['row_id'], test2['duration']], axis=1)
submission.to_csv('wait3.csv', index=False)

### just try nn for fun

In [22]:
from sklearn.neural_network import MLPRegressor

In [24]:
clf = MLPRegressor(
    hidden_layer_sizes=(10,),  activation='relu', solver='adam', alpha=0.001, batch_size='auto',
    learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
    random_state=0, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
    early_stopping=True, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

for _ in range(5): # 10 passes through the data
    #clf = GradientBoostingRegressor(warm_start = True, max_depth = 8, n_estimators = 200, random_state = 23)
    #clf = xgb.XGBRegressor(warm_start = True, max_depth = 10, n_estimators = 250, random_state = 23)
    train_iter = data_iter(90000, train_features, train_labels)
    train, valid = [], []

    print('==============epoch=================: ', _)
    
    
    for X, y in train_iter:
        #clf.fit(X,y)
        #print(X.shape)
        kf = KFold(n_splits=5, shuffle = True)

        for trn_idx, val_idx in kf.split(X):
        # split training data
            X_trn, X_tst, y_trn, y_tst = X[trn_idx], X[val_idx], y[trn_idx], y[val_idx] 

            clf.fit(X_trn, y_trn.ravel())
            trainpred = clf.predict(X_trn)
            
            train.append( mean_squared_error(y_trn, trainpred))

            pred = clf.predict(X_tst)
            #clf.n_estimators += 1 # increment by one so next  will add 1 tree
            valid.append(mean_squared_error(y_tst, pred))
    print('  Score train/valid: %.4f, %.4f' % (np.mean(np.sqrt(train)),np.mean(np.sqrt(valid))))

==============epoch=================:  0
  Score train/valid: 381.7383, 381.8613
==============epoch=================:  1
  Score train/valid: 381.0581, 381.2785
==============epoch=================:  2
  Score train/valid: 377.6057, 377.8959
==============epoch=================:  3
  Score train/valid: 376.1715, 376.0992
==============epoch=================:  4
  Score train/valid: 379.1709, 379.2503
